# Modelo Naive Bayes (Multinomial y Gaussiano)

En el presente notebook se construyen dos modelos Multinomial y Gaussiano a partir del conjunto de datos que sigue 
la estructura de la tabla mostrada abajo. Para esta tabla **blue**, **green**, **red**, **nir**, **swir1** y 
**swir2** son las características de los pixeles de una escena landsat y **wofs**, **bosque**, **wofs_bosque** y  
**ninguno** son las clases a la cual pertence casa pixel. Un pixel solo pertenece a una única clase.


| blue | green | red | nir | swir1 | swir2 | wofs | bosque | wofs_bosque | ninguno  |
|------|-------|-----|-----|-------|-------|------|--------|-------------|----------|
|      |       |     |     |       |       |      |        |             |          |


* El modelo generado para Multinomial es guardado en el archivo **model_nb_multinomial.joblib** 
* El modelo generado para Gaussiano es guardado en el archivo **model_nb_gaussiano.joblib** 

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn import datasets
from collections import Counter

from joblib import dump
from sklearn.externals import joblib

#from keras.models import model_from_json

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


/home/datacube/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Se importa el Dataset sin informacion de nubes
Como se ha descrito, las imagenes descargadas de los portales de imagenes satelitales presentan información en multiples bandas. Es con esta información que los algoritmos de clasificación pueden emitir un juicio para determinar a que tipo de información puede corresponder un pixel: bosque, no busque,nubes,agua, agua_bosque,ninguno.
En este escenario se plantea probar el clasificador Naive Bayes con información de imagenes que NO contengan nubes. Se plantea este escenario con el fin de determinar el rendmiento del modelo con un problema de este estilo.

In [2]:
satellite = pd.read_csv('https://docs.google.com/uc?export=download&id=1S_TUJMTQvSIKcPe8O3m_DXNFEMV4AURN')
satellite.head()

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno
0,272.0,172.0,74.0,61.0,81.0,62.0,1,0,0,0
1,172.0,158.0,71.0,62.0,107.0,92.0,1,0,0,0
2,172.0,104.0,14.0,1.0,44.0,39.0,1,0,0,0
3,255.0,202.0,103.0,87.0,128.0,104.0,1,0,0,0
4,255.0,197.0,100.0,86.0,113.0,92.0,1,0,0,0


# Filtro de valores negativos
Se realiza una filtro de las columnas que en alguna banda contengan un valor negativo porque el clasificador multinomial Naive bayes no funciona bien con esta información.

In [3]:
satellite = satellite[satellite["blue"] > 0]
satellite = satellite[satellite["red"] > 0]
satellite = satellite[satellite["green"] > 0]
satellite = satellite[satellite["nir"] > 0]
satellite = satellite[satellite["swir1"] > 0]
satellite = satellite[satellite["swir2"] > 0]

In [4]:
#satellite['wofs'].value_counts()
#satellite['bosque'].value_counts()
#satellite['wofs_bosque'].value_counts()
#satellite['cloud'].value_counts()
#satellite['ninguno'].value_counts()

# Se reemplaza los valores en columnas de clases
Como parte de preparación de los datos para crear el modelo se deben hacer algunos ajustes a la forma en que el dataset muestra la infiromación.Como se puede observar el dataset, las columnas de las clases se encuentran categorizadas por un valor binario. Lo que se realiza a continuación es reemplazar estos valores binarios por NaN en caso de ser "Cero" y el nombre de la clase en caso de ser "Uno".

In [5]:
# Reemplazar el valor de columna Wofs por NaN(0) y Wofs(1)
satellite['wofs'] = satellite['wofs'].replace(0,np.nan)
satellite['wofs'] = satellite['wofs'].replace(1,'wofs')


In [6]:
# Reemplazar el valor de columna bosque por NaN(0) y bosque(1)
satellite['bosque'] = satellite['bosque'].replace(0,np.nan)
satellite['bosque'] = satellite['bosque'].replace(1,'bosque')

In [7]:
# Reemplazar el valor de columna wofs_bosque por NaN(0) y wofs_bosque(1)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(0,np.nan)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(1,'wofs_bosque')

In [8]:
# Reemplazar el valor de columna ninguno por NaN(0) y ninguno(1)
satellite['ninguno'] = satellite['ninguno'].replace(0,np.nan)
satellite['ninguno'] = satellite['ninguno'].replace(1,'ninguno')

# Nueva columna de clasificación
Como ultima parte para la preparación de los datos debemos tener la clasificación en una sola columna. Procedemos a crearla con el nombre "label" y alli asignaremos el valor categorizado de cada conjunto de atributos.

In [9]:
#Se crea una nueva columna para almacenar el valor de etiqueta
satellite['label']=np.nan

#Se indica en la columna 'label', la clasificacion de la columna 'wofs'. Se esribe "Wofs" en columna 'label' si en  
#la columna 'wofs' esta este valor
for label in [col for col in satellite.columns if 'wofs' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'bosque'. Se esribe "bosque" en columna 'label' si en  
#la columna 'bosque' esta este valor
for label in [col for col in satellite.columns if 'bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'wofs_bosque'. Se esribe "wofs_bosque" en columna 'label' si en  
#la columna 'wofs_bosque' esta este valor
for label in [col for col in satellite.columns if 'wofs_bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'ninguno'. Se esribe "ninguno" en columna 'label' si en  
#la columna 'ninguno' esta este valor
for label in [col for col in satellite.columns if 'ninguno' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'cloud'. Se esribe "cloud" en columna 'label' si en  
#la columna 'cloud' esta este valor
for label in [col for col in satellite.columns if 'cloud' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
satellite.head(20)

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno,label
0,272.0,172.0,74.0,61.0,81.0,62.0,wofs,NaN,NaN,NaN,wofs
1,172.0,158.0,71.0,62.0,107.0,92.0,wofs,NaN,NaN,NaN,wofs
2,172.0,104.0,14.0,1.0,44.0,39.0,wofs,NaN,NaN,NaN,wofs
3,255.0,202.0,103.0,87.0,128.0,104.0,wofs,NaN,NaN,NaN,wofs
4,255.0,197.0,100.0,86.0,113.0,92.0,wofs,NaN,NaN,NaN,wofs
5,139.0,341.0,185.0,2804.0,1263.0,464.0,NaN,bosque,NaN,NaN,bosque
6,199.0,112.0,16.0,5.0,47.0,39.0,wofs,NaN,NaN,NaN,wofs
8,736.0,702.0,623.0,627.0,667.0,540.0,NaN,NaN,NaN,ninguno,ninguno
9,414.0,484.0,286.0,240.0,246.0,199.0,wofs,NaN,NaN,NaN,wofs
10,234.0,162.0,60.0,44.0,81.0,69.0,wofs,NaN,NaN,NaN,wofs


In [10]:
# Se crea el conjunto de calsificacion y el de entrenamiento
Y = satellite["label"]
# X = satellite.drop(columns=["label","wofs","bosque","wofs_bosque","cloud","ninguno"], axis=1)
X = satellite.drop(columns=["label","wofs","bosque","wofs_bosque","ninguno"], axis=1)
Y.value_counts()

wofs           709190
bosque         192531
ninguno         29327
wofs_bosque       476
Name: label, dtype: int64

# Se crean los conjuntos de entrenamiento y de prueba
Para hacer esto se deben dividir los datos en 4 conjuntos de datos de la siguiente manera: Conjuntos de entrenamiento(X_train, y_train) y Conjunto de test(X_test, y_test)

In [11]:
# Se dividen los datos en los conjuntos de Train y test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.3,
    random_state=0
)

In [12]:
X_test

,blue,green,red,nir,swir1,swir2
872806,220.0,128.0,36.0,22.0,59.0,51.0
224269,281.0,177.0,80.0,59.0,83.0,68.0
125685,388.0,360.0,257.0,247.0,253.0,199.0
360405,189.0,521.0,282.0,3839.0,1620.0,575.0
388185,301.0,245.0,153.0,137.0,157.0,131.0
...,...,...,...,...,...,...
64888,548.0,556.0,468.0,473.0,614.0,511.0
205796,341.0,247.0,146.0,125.0,153.0,111.0
908170,220.0,409.0,270.0,3094.0,1416.0,562.0
87597,242.0,246.0,136.0,124.0,159.0,134.0


In [13]:
y_test

872806       wofs
224269       wofs
125685       wofs
360405     bosque
388185       wofs
           ...   
64888     ninguno
205796       wofs
908170     bosque
87597        wofs
826113       wofs
Name: label, Length: 279458, dtype: object

# Naive Bayes para multiples capas
La clasificación multicapa del algoritmo Naive Bayes es conocida como "Clasificación Naive Bayes Multinomial". Como se puede apreciar la clasificación de nuestro conjunto de caracteristicas puede tener 4 valores: wofs, bosque, wofs_bosque y ninguno. 
En esta parte de creación del modelo se evaluaran dos variaciones del clasificador Naive Bayes: Gausiano y multinomial. En el analisis de resultados se contrastaran los resultados obtenidos para cada uno de ellos.

# Multinomial

In [14]:
#Create a Gaussian Classifier
gnb = MultinomialNB()

In [15]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [17]:
Counter(y_pred)

Counter({'wofs': 212227, 'bosque': 58302, 'ninguno': 8083, 'wofs_bosque': 846})

In [18]:
confusion_matrix(y_test, y_pred)

array([[ 57498,    235,      0,     80],
       [   803,   6680,   1210,     84],
       [     0,   1160, 210992,    576],
       [     1,      8,     25,    106]])

In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.99      0.99      0.99     57813
     ninguno       0.83      0.76      0.79      8777
        wofs       0.99      0.99      0.99    212728
 wofs_bosque       0.13      0.76      0.22       140

    accuracy                           0.99    279458
   macro avg       0.73      0.88      0.75    279458
weighted avg       0.99      0.99      0.99    279458



In [20]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9850353183662661


In [21]:
#toBePersisted = dict({
#    'metadata': {
#        'name': 'NB_Multinomial'
#    }
#})
#dump(toBePersisted, 'NB_Multinomial.joblib')

In [22]:
# joblib.dump(gnb, 'NB_Multinomial.joblib') 
joblib.dump(gnb, 'model_nb_multinomial.joblib') 

['model_nb_multinomial.joblib']

# Gaussiano

In [23]:
#Create a Gaussian Classifier
gnb = GaussianNB()

In [24]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [26]:
Counter(y_pred)

Counter({'wofs': 205895,
         'bosque': 54640,
         'ninguno': 15566,
         'wofs_bosque': 3357})

In [27]:
confusion_matrix(y_test, y_pred)

array([[ 54306,   1897,     13,   1597],
       [   320,   8068,    160,    229],
       [     8,   5578, 205658,   1484],
       [     6,     23,     64,     47]])

In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.99      0.94      0.97     57813
     ninguno       0.52      0.92      0.66      8777
        wofs       1.00      0.97      0.98    212728
 wofs_bosque       0.01      0.34      0.03       140

    accuracy                           0.96    279458
   macro avg       0.63      0.79      0.66    279458
weighted avg       0.98      0.96      0.97    279458



In [29]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9592818956694745


In [30]:
#toBePersisted = dict({
#    'metadata': {
#        'name': 'NB_Multinomial'
#    }
#})
#dump(toBePersisted, 'NB_Multinomial.joblib')

In [31]:
# joblib.dump(gnb, 'NB_Multinomial.joblib') 
joblib.dump(gnb, 'model_nb_gaussiano.joblib') 

['model_nb_gaussiano.joblib']

## Analisis de resultados y conclusiones

El rendmiento obtenido con el modelo Multinomial llega a ser del 98.50% y el rendimiento con el modelo Gausiano alcanza el 95.92%. Por lo que logramos determinar que el modelo Multinomial logra funcionar mejor por 3 puntos de diferencia. Revisando la matriz de confusión logramos determinar que la principal diferencia entre los modelos es que en el multinomial se confunde menos entre elementos de la clase 2(Ninguno) y la clase 3(WOFS - Agua).

**MultinomialGNB**

Accuracy: 98.50%

| Clase\Predicción | WOFS  | Bosque | Ninguno | WOFS_Bosque |
|------------------|-------|--------|---------|-------------|
| WOFS             | 57334 | 233    | 0       | 88          |
| Bosque           | 824   | 6422   | 1270    | 88          |
| Ninguno          | 0     | 1126   | 211310  | 514         |
| WOFS_Bosque      | 0     | 4      | 18      | 100         |

**GaussianMB**

Accuracy: 95.92%

| Clase\Predicción | WOFS  | Bosque | Ninguno | WOFS_Bosque |
|------------------|-------|--------|---------|-------------|
| WOFS             | 54063 | 1858   | 14      | 1720        |
| Bosque           | 298   | 7973   | 146     | 187         |
| Ninguno          | 12    | 5767   | 205946  | 1225        |
| WOFS_Bosque      | 2     | 24     | 52      | 44          |



El modelo Multinomial en terminos generales esta ofreciendo mejor rendimiento que el modelo Gaussiano. Esta diferencia es causada principalmente por el hecho que el modelo Gaussiano se equivoca mas para datos que se encuentran clasificados como Ningun y WOFS_bosque.
Por otro lado la clasificación “ninguno” puede estar presentando un bajo recall comparado con sus demas clases porque es una clasificación donde pueden estar una gran diversidad de valores haciendo que este modelo no generalice bien para esta clase particular.


# Referencias
https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

https://datascience.stackexchange.com/questions/40345/how-to-convert-multiple-columns-into-single-columns-in-pandas

https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra